# Neighborhood Computations

In [ ]:
import numpy as np
import igl
import meshplot

In [ ]:
bunny_v, bunny_f = igl.read_triangle_mesh("data/bunny.off")
cube_v, cube_f = igl.read_triangle_mesh("data/cube.obj")
sphere_v, sphere_f = igl.read_triangle_mesh("data/sphere.obj")
tube_v, tube_f = igl.read_triangle_mesh("data/bumpy.off")

In [ ]:
meshplot.offline()
meshplot.plot(bunny_v, bunny_f, shading={"wireframe": True})

In [ ]:
meshplot.plot(cube_v, cube_f, shading={"wireframe": True})

In [ ]:
meshplot.plot(sphere_v, sphere_f, shading={"wireframe": True})

## Vertex-to-Face Relations

In [ ]:
def vertex_to_face(V, F):
    (VF, NI) = igl.vertex_triangle_adjacency(F, np.size(V))
    face = 0
    vertex = 0
    vertex_index = 0
    while NI[vertex] < len(VF):
        print('The vertex', vertex_index, 'is adjacent to the faces ', end='')
        for j in range(NI[vertex + 1] - NI[vertex]):                        
            print(VF[face], ', ' , end='')
            face += 1
        print()
        vertex += 1
        vertex_index += 1

## Vertex-to-Vertex Relations

In [ ]:
def vertex_to_vertex(F):
    list = igl.adjacency_list(F)
    for vertex in range(len(list)):
        print('The vertex', vertex, 'is adjacent to the vertices ', end='')
        print(list[vertex])

## Visualizing the Neighborhood Relations

This section displays the vertex-to-face relations of the bunny object.

In [ ]:
vertex_to_face(bunny_v, bunny_f)

This section displays the vertex-to-vertex relations of the bunny object.

In [ ]:
vertex_to_vertex(bunny_f)

## Shading

Meshplot requires per vertex normals, so we need to "explode" the mesh.
Below section is to define a mesh explosion and helper function.

In [ ]:
def preprocess(v, f, key, thres = 50):

    if key == 2:
        n = igl.per_vertex_normals(v, f)
        return v, f, n    

    f_new = np.zeros(f.shape, dtype = int)
    v_new = np.zeros((f.shape[0] * 3, 3))
    v_index = 0

    for i in range(f.shape[0]):
        new_face = [v_index, v_index + 1, v_index + 2]
        f_new[i] = new_face
        v_new[v_index] = v[f[i][0]]
        v_new[v_index + 1] = v[f[i][1]]
        v_new[v_index + 2] = v[f[i][2]]
        v_index += 3

    if key == 1:
        n = igl.per_face_normals(v, f, np.array([0., 1., 0.]))
        n_new = np.zeros(v_new.shape)
        for i in range(f.shape[0]):
            f_normal = n[i]
            n_new[f_new[i][0]] = f_normal
            n_new[f_new[i][1]] = f_normal
            n_new[f_new[i][2]] = f_normal
        return v_new, f_new, n_new
    
    if key == 3:
        n = igl.per_corner_normals(v, f, thres)
        return v_new, f_new, n

### Flat Shading

Sphere.
For better visual effects of plotting, the default color is set in the meshplot function. Please comment on the line of code about color setting and pass it as argument of the meshplot function if you want to view the grayscale plotting. It applies to all following plots.

In [ ]:
v, f, n = preprocess(sphere_v, sphere_f, 1)
#color = np.full(np.shape(n), 1)
p = meshplot.plot(v, f, n = n, shading = {'flat': False, 'wireframe' : False})
p.add_lines(v, v + 0.1 * n)

Cube. Please note that the light source is directly incident to the front face of the cube, so rotate it around to see the shading effects. It applies to all following cube plots.

In [ ]:
v, f, n = preprocess(cube_v, cube_f, 1)
#color = np.full(np.shape(n), 1)
p = meshplot.plot(v, f, n = n, shading = {'flat': False, 'wireframe' : False})
p.add_lines(v, v + 0.1 * n)

### Per-vertex Shading

Sphere.

In [ ]:
v, f, n = preprocess(sphere_v, sphere_f, 2)
#color = np.full(np.shape(n), 1)
p = meshplot.plot(v, f, n = n, shading = {'flat': False, 'wireframe' : False})
p.add_lines(v, v + 0.1 * n)

Cube.

In [ ]:
v, f, n = preprocess(cube_v, cube_f, 2)
#color = np.full(np.shape(n), 1)
p = meshplot.plot(v, f, n = n, shading = {'flat': False, 'wireframe' : False})
p.add_lines(v, v + 0.1 * n)

### Per-corner Shading
The threshold argument of function per_corner_normals is set to 50 defautly. Please modify the value if you want to view different visual effects of per-corner shading.

Sphere. The smooth surface of the sphere could be clearly seen.

In [ ]:
v, f, n = preprocess(sphere_v, sphere_f, 3)
#color = np.full(np.shape(n), 1)
p = meshplot.plot(v, f, n = n, shading = {'flat': False, 'wireframe' : False})
p.add_lines(v, v + 0.1 * n)

## Connected Components
For better visual effects of connected components, I pour different and unique colors to the connected components of an object. The colors are sampled randomly.

In [ ]:
def connected_components(obj_address):
    v, f = igl.read_triangle_mesh(obj_address)
    com = igl.face_components(f)
    color_index = []
    for i in range(len(com)):
        if com[i] not in color_index:
            color_index.append(com[i])
    color_list = np.random.rand(len(color_index), 3)
    print('The number of connected components:')
    print(len(color_index))   
    size_list = np.zeros(len(color_index))
    for i in range(com.shape[0]):
        size_list[com[i]] += 1    
    for i in range(size_list.shape[0]):
        print('The size of component', i, 'is', size_list[i])    
    color_mat = np.zeros(np.shape(f))
    for i in range(np.shape(f)[0]):
        color_mat[i] = color_list[com[i]]
    meshplot.plot(v, f, c = color_mat)

Car.

In [ ]:
connected_components('data/car.off')

Teacup.

In [ ]:
connected_components('data/coffeecup.off')

## A simple subdivision scheme

In [ ]:
def subdivision(V, F):
#     meshplot.plot(V, F, shading={'wireframe': True})
#     F2 = np.zeros((F.shape[0] * 3, 3), dtype=int)
#     idx_max_v = V.shape[0]
#     for i in range(F.shape[0]):
#         F2[i * 3]     = np.array([F[i][0], F[i][1], idx_max_v + i])
#         F2[i * 3 + 1] = np.array([F[i][0], F[i][2], idx_max_v + i])
#         F2[i * 3 + 2] = np.array([F[i][1], F[i][2], idx_max_v + i])

    idx_max_v = len(V);
    F2 = []
    for i in range (len(F)):
        for j in range(3):
            temp = [F[i][j], F[i,(j+1)%3] , (idx_max_v + i)];
            F2.append(temp);
    F2 = np.array(F2)
    M = igl.barycenter(V, F)
    
    v_to_v_list = igl.adjacency_list(F)
    P = np.zeros(V.shape)
    for i in range(P.shape[0]):
        n = np.size(v_to_v_list[i])
        a_n = (4 - 2 * np.cos(2 * np.pi / n)) / 9
        sum = np.zeros(3)
        for j in range(n):
            sum += V[v_to_v_list[i][j]]
        P[i] = (1 - a_n) * V[i] + a_n / n * sum
    V1 = np.concatenate((P, M))
    #meshplot.plot(V1, F2, shading={'wireframe': True})

    TT = igl.triangle_triangle_adjacency(F)[0]
    F1 = [];
    for i in range (len(F)):
        temp = [];
        for j in range(3):
            temp = [F[i][j], len(V) + TT[i][j] , (len(V) + i)];
            F1.append(temp);
    F1 = np.array(F1)    
    
    return V1,F1

Bumpy. We could clearly observe that the tube object is not smooth when we load it to plot. But the subdivision method effectively smooths the mesh after processing.


In [ ]:
meshplot.plot(tube_v, tube_f, shading={'wireframe': True})
v1, f1 = subdivision(tube_v, tube_f)
meshplot.plot(v1, f1, shading={'wireframe': True})

Sphere. We could clearly observe that the sphere object is not smooth when we load it to plot. But the subdivision method effectively smooths the mesh after processing.


In [ ]:
meshplot.plot(sphere_v, sphere_f, shading={'wireframe': True})
v1, f1 = subdivision(sphere_v, sphere_f)
meshplot.plot(v1, f1, shading={'wireframe': True})